In [84]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOllama
import pandas as pd
import json
import re

## 提取出文本中的物品

In [85]:
# Temp = 0 so that we get clean information without a lot of creativity
chat_model = ChatOllama(temperature=0, model="qwen:32b-chat-v1.5-q5_K_M")

text_input = """
在这个宽敞的维多利亚风格卧室中，家具和陈设的精致细节彰显了19世纪的优雅与奢华。房间的中心是一张四柱大床，床头板雕刻着复杂的花卉图案，床尾则装饰着流苏的床尾凳，
增添了一丝贵族气息。床的两侧，是一对精美的床头柜，上面摆放着镀金的台灯和小巧的时钟，细节之处透露着维多利亚时代的精致。
房间的窗户朝南，阳光透过蕾丝窗帘洒入室内，为整个空间带来了温暖和明亮。窗户下方，一张软垫的长椅摆放在那里，是阅读或沉思的理想之地。
对面的墙上，是一面装饰着精美雕花的壁炉，即使在不使用的季节，它也是房间内不可或缺的装饰元素。
在房间的一角，摆放着一个小巧的书桌和书架，书架上整齐地排列着各种书籍和装饰品，书桌上则摆放着精致的文具和一盏台灯，为这个空间增添了一份学术氛围。
整个房间的布局既实用又美观，反映了维多利亚时代对于家居美学的追求。
"""

get_object_chain = ChatPromptTemplate.from_template("提取出如下文本中的陈设、家具和物品，输出一个数组，尽量详尽。文本：{text_input}") | chat_model | StrOutputParser()

objects_in_text = get_object_chain.invoke({"text_input": text_input})

print(objects_in_text)

['四柱大床', '床头板', '流苏的床尾凳', '床头柜', '镀金的台灯', '小巧的时钟', '窗户', '蕾丝窗帘', '软垫的长椅', '壁炉', '书桌', '书架', '书籍', '装饰品', '精致的文具', '台灯']


## 在素材库中找到匹配的物品

In [86]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品"),
    ResponseSchema(name="object_in_gallery", description="这是你觉得与用户输入最匹配的物品"),
    ResponseSchema(name="match_score", description="你认为用户输入的匹配与你匹配的接近程度的0-100分")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

template = """
您将获得用户提供的一系列物品名称。
在已入库物品列表中找到最合适的匹配项。
最接近的匹配将是具有最接近语义的匹配。不仅仅是字符串相似。

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
{objects_in_text}

已入库物品:
{objects_name_in_gallery}

你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["object_in_text", "objects_in_gallery"],
    partial_variables={"format_instructions": format_instructions}
)

def GetObjectsInGallery(csv_file_path):
    df = pd.read_csv(csv_file_path)
    objects = df['物品'].values
    objects_url = df['URL'].values
    objects_x_size = df['X轴尺寸'].values
    objects_y_size = df['Y轴尺寸'].values
    objects_z_size = df['Z轴尺寸'].values
    objects_scale = df['Scale'].values

    objects_in_gallery = []
    for i in range(len(objects)):
        obj = {'object_in_gallery':objects[i], 'objects_url_in_gallery':objects_url[i], 
        'objects_x_size':objects_x_size[i], 'objects_y_size':objects_y_size[i], 'objects_z_size':objects_z_size[i], 'objects_scale':objects_scale[i]}
        objects_in_gallery.append(obj)

    return objects_in_gallery

def GetObjectsNameInGallery(csv_file_path):
    df = pd.read_csv(csv_file_path)
    objects = ", ".join(df['物品'].values)
    return objects

objects_name_in_gallery = GetObjectsNameInGallery('../data/ObjectsInGallery_LayoutTest.csv')

# Your user input
_input = prompt.format_prompt(objects_in_text=objects_in_text, objects_name_in_gallery=objects_name_in_gallery)

print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

output = chat_model(_input.to_messages())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_in_gallery": string  // 这是你觉得与用户输入最匹配的物品
	"match_score": string  // 你认为用户输入的匹配与你匹配的接近程度的0-100分
}
```
There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

您将获得用户提供的一系列物品名称。
在已入库物品列表中找到最合适的匹配项。
最接近的匹配将是具有最接近语义的匹配。不仅仅是字符串相似。

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_in_gallery": string  // 这是你觉得与用户输入最匹配的物品
	"match_score": string  // 你认为用户输入的匹配与你匹配的接近程度的0-100分
}
```

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
['四柱大床', '床头板', '流苏的床尾凳', '床头柜', '镀金的台灯', '小巧的时钟', '窗户', '蕾丝窗帘', '软垫的长椅', '壁炉', '书桌', '书架', '书籍', '装饰品', '精致的文具', '台灯']

已入库物品:
仿古灯, 古董吊灯, 古董木制书柜, 圆桌和椅子, 吊灯, 镜子B, 维多利亚式控制台桌, 维多利亚风格沙发, 老爱

In [87]:
# 提取JSON内容
if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

if "```" in json_string:
    json_string = json_string.split("```")[0].strip()


# 去掉不符合格式的内容
# 使用正则表达式替换掉 // 和其后面的所有内容，直到行尾
cleaned_json_str = re.sub(r'//.*', '', json_string)

# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(cleaned_json_str)
objects_in_gallery = GetObjectsInGallery('../data/ObjectsInGallery_LayoutTest.csv')
print(objects_in_gallery)

objects_matched = []
for obj in structured_data:
    if int(obj['match_score']) < 50:
        continue

    for obj_g in objects_in_gallery:
        if obj_g['object_in_gallery'] == obj['object_in_gallery']:
            obj.update(obj_g)
            objects_matched.append(obj)
            break

print(objects_matched)

# 存储为JSON文件
with open("../data/ObjectsMatched.json", 'w') as f:
    json.dump(objects_matched, f, default=int, ensure_ascii=False, indent=4)

# 表格展示
pd.DataFrame(objects_matched)

[{'object_in_gallery': '仿古灯', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Antique_Lamp.usdz', 'objects_x_size': 50, 'objects_y_size': 150, 'objects_z_size': 50, 'objects_scale': 1.0}, {'object_in_gallery': '古董吊灯', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Antique_Chandelier.usdz', 'objects_x_size': 100, 'objects_y_size': 200, 'objects_z_size': 100, 'objects_scale': 1.0}, {'object_in_gallery': '古董木制书柜', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Antique_wooden_bookcase_-_Game_model.usdz', 'objects_x_size': 200, 'objects_y_size': 200, 'objects_z_size': 40, 'objects_scale': 1.0}, {'object_in_gallery': '圆桌和椅子', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Round_Table_And_Chairs.usdz', 'objects_x_size': 300, 'objects_y_size': 100, 'objects_z_size': 300, 'objects_scale': 1.0}, {'object_in_gallery': '吊灯', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Lamps.usdz', 'objects_x_size': 150, 'objects_y_si

,object_in_text,object_in_gallery,match_score,objects_url_in_gallery,objects_x_size,objects_y_size,objects_z_size,objects_scale
0,四柱大床,维多利亚风格床,85,D:/Home/Downloads/LayoutTestObjects/Victorian_...,150,100,200,1.00
1,床头板,维多利亚风格床,70,D:/Home/Downloads/LayoutTestObjects/Victorian_...,150,100,200,1.00
2,流苏的床尾凳,切斯特菲尔德沙发,50,D:/Home/Downloads/LayoutTestObjects/chesterfie...,200,60,100,1.00
3,床头柜,维多利亚式控制台桌,60,D:/Home/Downloads/LayoutTestObjects/Victorian_...,100,100,50,0.01
4,小巧的时钟,韦斯莱钟,90,D:/Home/Downloads/LayoutTestObjects/The_Weasle...,60,150,40,1.00
5,软垫的长椅,绿皮扶手椅,75,D:/Home/Downloads/LayoutTestObjects/Green_leat...,60,70,60,1.00
6,书桌,木质书架,55,D:/Home/Downloads/LayoutTestObjects/Wooden_boo...,120,200,40,1.00
7,书架,木质书架,100,D:/Home/Downloads/LayoutTestObjects/Wooden_boo...,120,200,40,1.00
8,装饰品,古董吊灯,65,D:/Home/Downloads/LayoutTestObjects/Antique_Ch...,100,200,100,1.00


## 根据文本确定物品方位和详细的相对位置

In [88]:
# 使用self-discover方法
from langchain import hub

select_prompt = hub.pull("superfhwl/self-discovery-select")
adapt_prompt = hub.pull("superfhwl/self-discovery-adapt")
structured_prompt = hub.pull("superfhwl/self-discovery-structure")
reasoning_prompt = hub.pull("superfhwl/self-discovery-reasoning")

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

select_chain = select_prompt | chat_model | StrOutputParser()
adapt_chain = adapt_prompt | chat_model | StrOutputParser()
structure_chain = structured_prompt | chat_model | StrOutputParser()
reasoning_chain = reasoning_prompt | chat_model | StrOutputParser()
overall_chain = (
    RunnablePassthrough.assign(selected_modules=select_chain)
    .assign(adapted_modules=adapt_chain)
    .assign(reasoning_structure=structure_chain)
    .assign(answer=reasoning_chain)
)

with open("../data/ReasoningModules.json", 'r') as f:
    reasoning_modules = json.load(f)

reasoning_modules_str = "\n".join(reasoning_modules)


# 读取JSON数据
with open('../data/ObjectsMatched.json', 'r') as f:
    ObjectsMatched = json.load(f)

# 根据匹配信息整理输入布局的数据
objects_matched = []
for obj in ObjectsMatched:
    if int(obj['match_score']) > 0:
        obj_mat = {'物品名称':obj['object_in_text'], 
                    'X轴尺寸':obj['objects_x_size'], 
                    'Y轴尺寸':obj['objects_y_size'], 
                    'Z轴尺寸':obj['objects_z_size']}
        objects_matched.append(obj_mat)


task = f"""
您将获得用户提供的一段文本，描述了房间中的物品和陈设。同时还有一组用户输入的物品名称，它们与文本中的物品和陈设对应。同时还有物品的 X、Y、Z轴尺寸，以厘米为单位。
分析并输出一个物品和陈设的布局方案，应符合文本描述的内容。同时要符合人们日常生活的习惯，考虑生活便利性。物品和物品之间不要出现重叠和碰撞，间距和朝向要合理。
X轴指向东、Y轴指向上、Z轴指向南。场景中心坐标为 X=0,Y=0,Z=0。场景中心朝向为南，场景尺寸为X=800,Y=500,Z=800。物品朝向的角度：南=0、东=90、北=180、西=270

用户输入的物品信息:
{objects_matched}

用户提供的一段文本:
{text_input}

布局方案的格式：
物品：位置(X、Y、Z轴坐标)，朝向的角度
"""

output = overall_chain.invoke(
    {"task_description": task, "reasoning_modules": reasoning_modules_str}
)

for k in output:
    print(k)
    print('==============')
    print(output[k])

task_description

您将获得用户提供的一段文本，描述了房间中的物品和陈设。同时还有一组用户输入的物品名称，它们与文本中的物品和陈设对应。同时还有物品的 X、Y、Z轴尺寸，以厘米为单位。
分析并输出一个物品和陈设的布局方案，应符合文本描述的内容。同时要符合人们日常生活的习惯，考虑生活便利性。物品和物品之间不要出现重叠和碰撞，间距和朝向要合理。
X轴指向东、Y轴指向上、Z轴指向南。场景中心坐标为 X=0,Y=0,Z=0。场景中心朝向为南，场景尺寸为X=800,Y=500,Z=800。物品朝向的角度：南=0、东=90、北=180、西=270

用户输入的物品信息:
[{'物品名称': '四柱大床', 'X轴尺寸': 150, 'Y轴尺寸': 100, 'Z轴尺寸': 200}, {'物品名称': '床头板', 'X轴尺寸': 150, 'Y轴尺寸': 100, 'Z轴尺寸': 200}, {'物品名称': '流苏的床尾凳', 'X轴尺寸': 200, 'Y轴尺寸': 60, 'Z轴尺寸': 100}, {'物品名称': '床头柜', 'X轴尺寸': 100, 'Y轴尺寸': 100, 'Z轴尺寸': 50}, {'物品名称': '小巧的时钟', 'X轴尺寸': 60, 'Y轴尺寸': 150, 'Z轴尺寸': 40}, {'物品名称': '软垫的长椅', 'X轴尺寸': 60, 'Y轴尺寸': 70, 'Z轴尺寸': 60}, {'物品名称': '书桌', 'X轴尺寸': 120, 'Y轴尺寸': 200, 'Z轴尺寸': 40}, {'物品名称': '书架', 'X轴尺寸': 120, 'Y轴尺寸': 200, 'Z轴尺寸': 40}, {'物品名称': '装饰品', 'X轴尺寸': 100, 'Y轴尺寸': 200, 'Z轴尺寸': 100}]

用户提供的一段文本:

在这个宽敞的维多利亚风格卧室中，家具和陈设的精致细节彰显了19世纪的优雅与奢华。房间的中心是一张四柱大床，床头板雕刻着复杂的花卉图案，床尾则装饰着流苏的床尾凳，
增添了一丝贵族气息。床的两侧，是一对精美的床头柜，上面摆放着镀金的台灯和小巧的时钟，细节之处透露着维多利亚时代的精致。
房间的窗户朝南，阳光透过蕾丝窗帘洒入室内，为整个空间带来了温暖和明亮。窗户下方，一张软垫的长椅摆放在那里，

## 根据方位和相对位置，转换为坐标

In [89]:

response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品名称"),
    ResponseSchema(name="object_name", description="这是用户输入的物品的英语翻译（合法的C语言变量名称形式）"),
    ResponseSchema(name="translation_x", description="这是物品的位置X轴坐标，单位是厘米"),
    ResponseSchema(name="translation_y", description="这是物品的位置Y轴坐标，单位是厘米"),
    ResponseSchema(name="translation_z", description="这是物品的位置Z轴坐标，单位是厘米"),
    ResponseSchema(name="rotation_y", description="这是朝向角度（南=0、东=90、北=180、西=270）")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

template = """
您将获得一段用户输入的文本，描述了物品的位置(X、Y、Z轴坐标)，朝向的角度（以下数字之一：0、90、180、270）。

根据上述信息输出：

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品信息:
{text_input}

你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["text_input"],
    partial_variables={"format_instructions": format_instructions}
)

# Your user input
_input = prompt.format_prompt(text_input=output['answer'])


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

output = chat_model(_input.to_messages())

if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

if "```" in json_string:
    json_string = json_string.split("```")[0].strip()

print(json_string)

# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(json_string)

# 存储为JSON文件
with open("../data/ObjectsLayout.json", 'w') as f:
    json.dump(structured_data, f, ensure_ascii=False, indent=4)

# 表格展示
pd.DataFrame(structured_data)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品名称
	"object_name": string  // 这是用户输入的物品的英语翻译（合法的C语言变量名称形式）
	"translation_x": string  // 这是物品的位置X轴坐标，单位是厘米
	"translation_y": string  // 这是物品的位置Y轴坐标，单位是厘米
	"translation_z": string  // 这是物品的位置Z轴坐标，单位是厘米
	"rotation_y": string  // 这是朝向角度（南=0、东=90、北=180、西=270）
}
```
There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

您将获得一段用户输入的文本，描述了物品的位置(X、Y、Z轴坐标)，朝向的角度（以下数字之一：0、90、180、270）。

根据上述信息输出：

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品名称
	"object_name": string  // 这是用户输入的物品的英语翻译（合法的C语言变量名称形式）
	"translation_x": string  // 这是物品的位置X轴坐标，单位是厘米
	"translation_y": string  // 这是物品的位置Y轴坐标，单位是厘米
	"translation_z": string  // 这

,object_in_text,object_name,translation_x,translation_y,translation_z,rotation_y
0,四柱大床,four_poster_bed,0,200,0,0
1,床头板,headboard,-75,180,0,0
2,流苏的床尾凳,fringed_bedfoot_stool,75,180,0,0
3,床头柜,nightstand,-125,200,-25,0
4,床头柜,nightstand_2,125,200,-25,0
5,小巧的时钟,small_clock,-175,200,-25,0
6,软垫的长椅,cushioned_chair,0,350,0,90
7,书桌,desk,-200,0,-100,0
8,书架,bookshelf,-200,0,-60,0
9,装饰品,decoration,-200,0,-40,0


## 将JSON文件转换为USDA文件

In [91]:
import json
import os
from pxr import Usd, UsdGeom, Sdf, Gf

# 定义一个函数来创建USD对象
def create_usd_object(stage, prim_path, obj_name, obj_url, position, rotation):
    # 创建一个新的Prim在给定的路径上
    prim = stage.DefinePrim(prim_path, 'Xform')
    prim.name = obj_name

    xformable = UsdGeom.Xformable(prim)

    xformable.AddTranslateOp().Set(value=translation)
    xformable.AddRotateXYZOp().Set(value=rotation)

    # 创建payload，并设置其references指向目标USD文件
    payload = Sdf.Payload(obj_url)
    prim.GetPayloads().AddPayload(payload)

    print(xformable)
    
    return prim


# 创建USD stage
stage = Usd.Stage.CreateNew('../data/link_room_layout_2.usda')

# 创建一个新的Prim，这将作为默认Prim
default_prim = stage.DefinePrim('/Objects')

# 使用SetDefaultPrim方法设置默认Prim
stage.SetDefaultPrim(default_prim)

# 读取JSON数据
with open('../data/ObjectsMatched.json', 'r') as f:
    ObjectsMatched = json.load(f)

with open('../data/ObjectsLayout.json', 'r') as f:
    ObjectsLayout = json.load(f)

# # 合并两个JSON文件的数据
for obj in ObjectsMatched:
    if int(obj['match_score']) <= 0:
        continue

    for obj_layout in ObjectsLayout:
        if obj_layout['object_in_text'] == obj['object_in_text']:
            obj_layout.update(obj)

for o in ObjectsLayout:
    print(o)

for o in ObjectsMatched:
    print(o)

# 遍历JSON数据，为每个对象创建一个USD prim
for obj in ObjectsLayout:
    if not 'objects_url_in_gallery' in obj:
        continue

    prim_path = '/Objects/' + obj['object_name']
    translation = Gf.Vec3f(float(obj['translation_x']), float(obj['translation_y']), float(obj['translation_z']))
    rotation = Gf.Vec3f(0, float(obj['rotation_y']), 0)
    create_usd_object(stage, prim_path, obj['object_in_text'], obj['objects_url_in_gallery'], translation, rotation)

# 保存USD stage
stage.Save()


{'object_in_text': '四柱大床', 'object_name': 'four_poster_bed', 'translation_x': '0', 'translation_y': '200', 'translation_z': '0', 'rotation_y': '0', 'object_in_gallery': '维多利亚风格床', 'match_score': '85', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Victorian_Bed.usdz', 'objects_x_size': 150, 'objects_y_size': 100, 'objects_z_size': 200, 'objects_scale': 1.0}
{'object_in_text': '床头板', 'object_name': 'headboard', 'translation_x': '-75', 'translation_y': '180', 'translation_z': '0', 'rotation_y': '0', 'object_in_gallery': '维多利亚风格床', 'match_score': '70', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Victorian_Bed.usdz', 'objects_x_size': 150, 'objects_y_size': 100, 'objects_z_size': 200, 'objects_scale': 1.0}
{'object_in_text': '流苏的床尾凳', 'object_name': 'fringed_bedfoot_stool', 'translation_x': '75', 'translation_y': '180', 'translation_z': '0', 'rotation_y': '0', 'object_in_gallery': '切斯特菲尔德沙发', 'match_score': '50', 'objects_url_in_gallery': 'D:/Home/Downloa